In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with the Live API in Vertex AI using WebSockets


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/multimodal-live-api/intro_multimodal_live_api.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fmultimodal-live-api%2Fintro_multimodal_live_api.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/multimodal-live-api/intro_multimodal_live_api.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/multimodal-live-api/intro_multimodal_live_api.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/multimodal-live-api/intro_multimodal_live_api.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/multimodal-live-api/intro_multimodal_live_api.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/multimodal-live-api/intro_multimodal_live_api.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/multimodal-live-api/intro_multimodal_live_api.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/multimodal-live-api/intro_multimodal_live_api.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>


| | |
|-|-|
| Author(s) |  [Eric Dong](https://github.com/gericdong), [Holt Skinner](https://github.com/holtskinner) |

## Overview

The Live API enables low-latency bidirectional voice and video interactions with Gemini. The API can process text, audio, and video input, and it can provide text and audio output. 

See the [Live API](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-live) page for more details.

This tutorial demonstrates the following simple examples to help you get started with the Live API in Vertex AI using [WebSockets](https://en.wikipedia.org/wiki/WebSocket).

- Using Gemini 2.0 Flash
  - Text-to-text generation
  - Text-to-audio generation
  - Text-to-audio conversation
  - Function calling
  - Code execution
  - Audio transcription
  - Voice Activity Detection (VAD)
- Using Gemini 2.5 Flash native audio dialog
  - Proactive audio
  - Affective dialog

## Getting Started

### Install libraries

In [1]:
%pip install --upgrade --quiet websockets

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [3]:
PROJECT_ID = "[your-project-id]"  # @param {type: "string"}
LOCATION = "us-central1"  # @param {type: "string"}

HOST = "us-central1-aiplatform.googleapis.com"
SERVICE_URL = (
    f"wss://{HOST}/ws/google.cloud.aiplatform.v1.LlmBidiService/BidiGenerateContent"
)

## Generate an access token

`gcloud auth application-default print-access-token` generates and prints an access token for the current Application Default Credential. The default access token lifetime is 3600 seconds. This access token will be used to connect to the WebSocket server.

In [25]:
bearer_token = !gcloud auth application-default print-access-token

### Import libraries


In [27]:
import base64
import json
from pathlib import Path

import numpy as np
from IPython.display import Audio, Markdown, display
from websockets.asyncio.client import connect

## Using the Gemini 2.0 Flash

Live API is a new capability introduced with the [Gemini 2.0 Flash model](https://cloud.google.com/vertex-ai/generative-ai/docs/gemini-v2).

In [26]:
MODEL_ID = "gemini-2.0-flash-live-preview-04-09"  # @param {type: "string"}

MODEL = (
    f"projects/{PROJECT_ID}/locations/{LOCATION}/publishers/google/models/{MODEL_ID}"
)

### **Example 1**: Text-to-text generation

You send a text prompt and receive a text message.

**Notes**
- A session `ws` represents a single WebSocket connection between the client and the server.
- After a new connection is initiated, the session can exchange messages with the server.
- Messages are JSON-formatted strings exchanged over the WebSocket connection.
- The first message to be sent should be a [`setup`](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-live#bidigeneratecontentsetup) message that contains the model, generation parameters, system instructions, and tools.
  - `response_modalities` accepts `TEXT` or `AUDIO`.
- To receive messages from the server, listen for the WebSocket `message` event, and then parse the result according to the definition of supported server messages.


In [28]:
# Set model generation_config
CONFIG = {"response_modalities": ["TEXT"]}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {bearer_token[0]}",
}

# Connect to the server
async with connect(SERVICE_URL, additional_headers=headers) as ws:
    # Set up the session
    await ws.send(
        json.dumps(
            {
                "setup": {
                    "model": MODEL,
                    "generation_config": CONFIG,
                }
            }
        )
    )

    # Receive setup response
    raw_response = await ws.recv(decode=False)
    setup_response = json.loads(raw_response.decode("ascii"))

    # Send text message
    text_input = "Hello? Gemini are you there?"
    display(Markdown(f"**Input:** {text_input}"))

    msg = {
        "client_content": {
            "turns": [{"role": "user", "parts": [{"text": text_input}]}],
            "turn_complete": True,
        }
    }

    await ws.send(json.dumps(msg))

    responses = []

    # Receive chucks of server response
    async for raw_response in ws:
        response = json.loads(raw_response.decode())
        server_content = response.pop("serverContent", None)
        if server_content is None:
            break

        model_turn = server_content.pop("modelTurn", None)
        if model_turn is not None:
            parts = model_turn.pop("parts", None)
            if parts is not None:
                responses.append(parts[0]["text"])

        # End of turn
        turn_complete = server_content.pop("turnComplete", None)
        if turn_complete:
            break

    # Print the server response
    display(Markdown(f"**Response >** {''.join(responses)}"))

**Input:** Hello? Gemini are you there?

**Response >** Yes, I'm here. What's on your mind?


### **Example 2**: Text-to-audio generation

You send a text prompt and receive a model response in audio.

**Notes**
- The Live API supports the following voices:
  - Puck
  - Charon
  - Kore
  - Fenrir
  - Aoede
- To specify a voice, set the `voice_name` within the `speech_config` object, as part of your session configuration.
- To specify a language , set the `language_code` within the `speech_config` as part of your session configuration. See the [supported languages](https://cloud.google.com/text-to-speech/docs/list-voices-and-types#list_of_all_supported_languages).


In [17]:
# Set model generation_config
CONFIG = {
    "response_modalities": ["AUDIO"],
    "speech_config": {
        "voice_config": {"prebuilt_voice_config": {"voice_name": "Aoede"}},
        "language_code": "es-ES",
    },
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {bearer_token[0]}",
}

# Connect to the server
async with connect(SERVICE_URL, additional_headers=headers) as ws:
    # Set up the session
    await ws.send(
        json.dumps(
            {
                "setup": {
                    "model": MODEL,
                    "generation_config": CONFIG,
                }
            }
        )
    )

    # Receive setup response
    raw_response = await ws.recv(decode=False)
    setup_response = json.loads(raw_response.decode("ascii"))

    # Send text message
    text_input = "Hello? Gemini are you there?"
    display(Markdown(f"**Input:** {text_input}"))

    msg = {
        "client_content": {
            "turns": [{"role": "user", "parts": [{"text": text_input}]}],
            "turn_complete": True,
        }
    }

    await ws.send(json.dumps(msg))

    responses = []

    # Receive chucks of server response
    async for raw_response in ws:
        response = json.loads(raw_response.decode())
        server_content = response.pop("serverContent", None)
        if server_content is None:
            break

        model_turn = server_content.pop("modelTurn", None)
        if model_turn is not None:
            parts = model_turn.pop("parts", None)
            if parts is not None:
                for part in parts:
                    pcm_data = base64.b64decode(part["inlineData"]["data"])
                    responses.append(np.frombuffer(pcm_data, dtype=np.int16))

        # End of turn
        turn_complete = server_content.pop("turnComplete", None)
        if turn_complete:
            break

    # Play the returned audio message
    display(Audio(np.concatenate(responses), rate=24000, autoplay=True))

**Input:** Hello? Gemini are you there?

### **Example 3**: Text-to-audio conversation

**Step 1**: You set up a conversation with the API that allows you to send text prompts and receive audio responses.

**Notes**

- While the model keeps track of in-session interactions, explicit session history accessible through the API isn't available yet. When a session is terminated the corresponding context is erased.


In [18]:
# Set model generation_config
CONFIG = {"response_modalities": ["AUDIO"]}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {bearer_token[0]}",
}


async def main() -> None:
    # Connect to the server
    async with connect(SERVICE_URL, additional_headers=headers) as ws:
        # Set up the session
        async def setup() -> None:
            await ws.send(
                json.dumps(
                    {
                        "setup": {
                            "model": MODEL,
                            "generation_config": CONFIG,
                        }
                    }
                )
            )

            # Receive setup response
            raw_response = await ws.recv(decode=False)
            setup_response = json.loads(raw_response.decode("ascii"))
            print(f"Connected: {setup_response}")

        # Send text message
        async def send() -> bool:
            text_input = input("Input > ")
            if text_input.lower() in ("q", "quit", "exit"):
                return False

            msg = {
                "client_content": {
                    "turns": [{"role": "user", "parts": [{"text": text_input}]}],
                    "turn_complete": True,
                }
            }

            await ws.send(json.dumps(msg))
            return True

        # Receive server response
        async def receive() -> None:
            responses = []

            # Receive chucks of server response
            async for raw_response in ws:
                response = json.loads(raw_response.decode())
                server_content = response.pop("serverContent", None)
                if server_content is None:
                    break

                model_turn = server_content.pop("modelTurn", None)
                if model_turn is not None:
                    parts = model_turn.pop("parts", None)
                    if parts is not None:
                        for part in parts:
                            pcm_data = base64.b64decode(part["inlineData"]["data"])
                            responses.append(np.frombuffer(pcm_data, dtype=np.int16))

                # End of turn
                turn_complete = server_content.pop("turnComplete", None)
                if turn_complete:
                    break

            # Play the returned audio message
            display(Markdown("**Response >**"))
            display(Audio(np.concatenate(responses), rate=24000, autoplay=True))

        await setup()

        while True:
            if not await send():
                break
            await receive()

**Step 2** Start the conversation, input your prompts, or type `q`, `quit` or `exit` to exit.


In [19]:
await main()

Connected: {'setupComplete': {}}
Input > Hello


**Response >**

Input > q


### **Example 4**: Function calling

You can use function calling to create a description of a function, then pass that description to the model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with.

**Notes**:

- All functions must be declared at the start of the session by sending tool definitions as part of the `setup` message.
- Currently only one tool is supported in the API.

In [20]:
# Set model generation_config
CONFIG = {"response_modalities": ["TEXT"]}

# Define function declarations
TOOLS = {
    "function_declarations": {
        "name": "get_current_weather",
        "description": "Get the current weather in the given location",
        "parameters": {
            "type": "OBJECT",
            "properties": {"location": {"type": "STRING"}},
        },
    }
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {bearer_token[0]}",
}

# Connect to the server
async with connect(SERVICE_URL, additional_headers=headers) as ws:
    # Set up the session
    await ws.send(
        json.dumps(
            {
                "setup": {
                    "model": MODEL,
                    "generation_config": CONFIG,
                    "tools": TOOLS,
                }
            }
        )
    )

    # Receive setup response
    raw_response = await ws.recv(decode=False)
    setup_response = json.loads(raw_response.decode())

    # Send text message
    text_input = "Get the current weather in Santa Clara, San Jose and Mountain View"
    display(Markdown(f"**Input:** {text_input}"))

    msg = {
        "client_content": {
            "turns": [{"role": "user", "parts": [{"text": text_input}]}],
            "turn_complete": True,
        }
    }

    await ws.send(json.dumps(msg))

    responses = []

    # Receive chucks of server response
    async for raw_response in ws:
        response = json.loads(raw_response.decode("UTF-8"))

        if (tool_call := response.get("toolCall")) is not None:
            for function_call in tool_call["functionCalls"]:
                responses.append(f"FunctionCall: {function_call!s}\n")

        if (server_content := response.get("serverContent")) is not None:
            if server_content.get("turnComplete", True):
                break

    # Print the server response
    display(Markdown("**Response >** {}".format("\n".join(responses))))

**Input:** Get the current weather in Santa Clara, San Jose and Mountain View

**Response >** FunctionCall: {'name': 'get_current_weather', 'args': {'location': 'Santa Clara'}}

FunctionCall: {'name': 'get_current_weather', 'args': {'location': 'San Jose'}}

FunctionCall: {'name': 'get_current_weather', 'args': {'location': 'Mountain View'}}


### **Example 5**: Code execution

You can use code execution capability to generate and execute Python code directly within the API.

In this example, you initialize the code execution tool by passing `code_execution` in the `Tools` configuration, and register this tool with the model at the start of the session by sending tool definitions as part of the `setup` message.

In [21]:
# Set model generation_config
CONFIG = {"response_modalities": ["TEXT"]}

# Set code execution
TOOLS = {"code_execution": {}}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {bearer_token[0]}",
}

# Connect to the server
async with connect(SERVICE_URL, additional_headers=headers) as ws:
    # Set up the session
    await ws.send(
        json.dumps(
            {
                "setup": {
                    "model": MODEL,
                    "generation_config": CONFIG,
                    "tools": TOOLS,
                }
            }
        )
    )

    # Receive setup response
    raw_response = await ws.recv(decode=False)
    setup_response = json.loads(raw_response.decode())

    # Send text message
    text_input = "Write code to calculate the 15th fibonacci number then find the nearest palindrome to it"
    display(Markdown(f"**Input:** {text_input}"))

    msg = {
        "client_content": {
            "turns": [{"role": "user", "parts": [{"text": text_input}]}],
            "turn_complete": True,
        }
    }

    await ws.send(json.dumps(msg))

    responses = []

    # Receive chucks of server response
    async for raw_response in ws:
        response = json.loads(raw_response.decode("UTF-8"))

        if (server_content := response.get("serverContent")) is not None:
            if (model_turn := server_content.get("modelTurn")) is not None:
                if (parts := model_turn.get("parts")) is not None:
                    if parts[0].get("text"):
                        responses.append(parts[0]["text"])
                    for part in parts:
                        if (executable_code := part.get("executableCode")) is not None:
                            display(
                                Markdown(
                                    f"""**Executable code:**
```py
{executable_code.get("code")}
```
                            """
                                )
                            )
            if server_content.get("turnComplete", False):
                break

    # Print the server response
    display(Markdown(f"**Response >** {''.join(responses)}"))

**Input:** Write code to calculate the 15th fibonacci number then find the nearest palindrome to it

**Executable code:**
```py
def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n + 1):
            a, b = b, a + b
        return b

def nearest_palindrome(n):
    s = str(n)
    l = len(s)
    mid = l // 2
    left = s[:mid]
    
    if l % 2 == 0:
        palindrome1 = left + left[::-1]
    else:
        palindrome1 = left + s[mid] + left[::-1]
    
    palindrome1_int = int(palindrome1)
    
    if palindrome1_int == n:
        return n
    
    if l % 2 == 0:
        left_int = int(left)
        left_minus = str(left_int - 1)
        palindrome2 = left_minus + left_minus[::-1]
        
        left_plus = str(left_int + 1)
        palindrome3 = left_plus + left_plus[::-1]
    else:
        middle = int(s[mid])
        left_int = int(left)
        
        palindrome2 = left + str(middle-1 if middle > 0 else 9) + left[::-1]
        
        left_middle_plus = left + str(middle + 1)
        
        palindrome3 = left_middle_plus + left_middle_plus[:len(left)][::-1]
        
    palindrome2_int = int(palindrome2)
    palindrome3_int = int(palindrome3)

    
    
    diff1 = abs(n - palindrome1_int)
    diff2 = abs(n - palindrome2_int)
    diff3 = abs(n - palindrome3_int)
    
    
    
    if diff1 <= diff2 and diff1 <= diff3:
        return palindrome1_int
    elif diff2 <= diff1 and diff2 <= diff3:
        return palindrome2_int
    else:
        return palindrome3_int

fib_number = fibonacci(15)
print(f'{fib_number=}')
nearest_pal = nearest_palindrome(fib_number)
print(f'{nearest_pal=}')


```
                            

**Response >** Okay, I can do that. First, I'll calculate the 15th Fibonacci number. Then I'll find the nearest palindrome to it. I'll use a Python tool to handle the Fibonacci calculation and palindrome search efficiently.

The 15th Fibonacci number is 610. The nearest palindrome to 610 is 606.


### **Example 6**: Google Search

The `google_search` tool lets the model conduct Google searches. For example, try asking it about events that are too recent to be in the training data.

In [22]:
# Set model generation_config
CONFIG = {"response_modalities": ["TEXT"]}

# Set google search
TOOLS = {"google_search": {}}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {bearer_token[0]}",
}

# Connect to the server
async with connect(SERVICE_URL, additional_headers=headers) as ws:
    # Set up the session
    await ws.send(
        json.dumps(
            {
                "setup": {
                    "model": MODEL,
                    "generation_config": CONFIG,
                    "tools": TOOLS,
                }
            }
        )
    )

    # Receive setup response
    raw_response = await ws.recv(decode=False)
    setup_response = json.loads(raw_response.decode())

    # Send text message
    text_input = "What is the current weather in San Jose, CA?"
    display(Markdown(f"**Input:** {text_input}"))

    msg = {
        "client_content": {
            "turns": [{"role": "user", "parts": [{"text": text_input}]}],
            "turn_complete": True,
        }
    }

    await ws.send(json.dumps(msg))

    responses = []

    # Receive chucks of server response
    async for raw_response in ws:
        response = json.loads(raw_response.decode())
        server_content = response.pop("serverContent", None)
        if server_content is None:
            break

        model_turn = server_content.pop("modelTurn", None)
        if model_turn is not None:
            parts = model_turn.pop("parts", None)
            if parts is not None:
                responses.append(parts[0]["text"])

        # End of turn
        turn_complete = server_content.pop("turnComplete", None)
        if turn_complete:
            break

    # Print the server response
    display(Markdown("**Response >** {}".format("\n".join(responses))))

**Input:** What is the current weather in San Jose, CA?

**Response >** The
 weather
 in San Jose, CA is currently clear with a temperature of 52°
F (11°C), but feels like 52°F (
11°C). The humidity is around 77% and there is a 0% chance of rain.


### **Example 7**: Audio transcription

The Live API provides transcriptions for both input and output audio.

In [23]:
# Set model generation_config
CONFIG = {
    "response_modalities": ["AUDIO"],
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {bearer_token[0]}",
}

# Connect to the server
async with connect(SERVICE_URL, additional_headers=headers) as ws:
    # Set up the session
    await ws.send(
        json.dumps(
            {
                "setup": {
                    "model": MODEL,
                    "generation_config": CONFIG,
                    "input_audio_transcription": {},
                    "output_audio_transcription": {},
                }
            }
        )
    )

    # Receive setup response
    raw_response = await ws.recv(decode=False)
    setup_response = json.loads(raw_response.decode("ascii"))

    # Send text message
    text_input = "Hello? Gemini are you there?"
    display(Markdown(f"**Input:** {text_input}"))

    msg = {
        "client_content": {
            "turns": [{"role": "user", "parts": [{"text": text_input}]}],
            "turn_complete": True,
        }
    }

    await ws.send(json.dumps(msg))

    responses = []
    input_transcriptions = []
    output_transcriptions = []

    # Receive chucks of server response
    async for raw_response in ws:
        response = json.loads(raw_response.decode())
        server_content = response.pop("serverContent", None)
        if server_content is None:
            break

        if (
            input_transcription := server_content.get("inputTranscription")
        ) is not None:
            if (text := input_transcription.get("text")) is not None:
                input_transcriptions.append(text)
        if (
            output_transcription := server_content.get("outputTranscription")
        ) is not None:
            if (text := output_transcription.get("text")) is not None:
                output_transcriptions.append(text)

        model_turn = server_content.pop("modelTurn", None)
        if model_turn is not None:
            parts = model_turn.pop("parts", None)
            if parts is not None:
                for part in parts:
                    pcm_data = base64.b64decode(part["inlineData"]["data"])
                    responses.append(np.frombuffer(pcm_data, dtype=np.int16))

        # End of turn
        turn_complete = server_content.pop("turnComplete", None)
        if turn_complete:
            break

    if input_transcriptions:
        display(Markdown(f"**Input transcription >** {''.join(input_transcriptions)}"))

    if responses:
        # Play the returned audio message
        display(Audio(np.concatenate(responses), rate=24000, autoplay=True))

    if output_transcriptions:
        display(
            Markdown(f"**Output transcription >** {''.join(output_transcriptions)}")
        )

**Input:** Hello? Gemini are you there?

**Output transcription >** Yep, I'm here. What's on your mind?

### **Example 8**: Voice Activity Detection (VAD)

Voice Activity Detection (VAD) allows the model to recognize when a person is speaking. This is essential for creating natural conversations, as it allows a user to interrupt the model at any time.

- By default, the model automatically performs voice activity detection on a continuous audio input stream. Voice activity detection can be configured with the `realtime_input_config.automatic_activity_detection` field of the `setup` message.
- When voice activity detection detects an interruption, the ongoing generation is canceled and discarded. Only the information already sent to the client is retained in the session history. The server then sends a message to report the interruption.
- When the audio stream is paused for more than a second (for example, because the user switched off the microphone), an `audioStreamEnd` event should be sent to flush any cached audio. The client can resume sending audio data at any time.

In [ ]:
# Download an example audio file
URL = "https://storage.googleapis.com/cloud-samples-data/generative-ai/audio/hello_are_you_there.pcm"
!wget -q $URL -O sample.pcm

In [ ]:
CONFIG = {
    "response_modalities": ["TEXT"],
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {bearer_token[0]}",
}

# Connect to the server
async with connect(SERVICE_URL, additional_headers=headers) as ws:
    # Set up the session
    await ws.send(
        json.dumps(
            {
                "setup": {
                    "model": MODEL,
                    "generation_config": CONFIG,
                    "realtime_input_config": {
                        "automatic_activity_detection": {
                            "disabled": False,  # default
                            "start_of_speech_sensitivity": "START_SENSITIVITY_HIGH",
                            "end_of_speech_sensitivity": "END_SENSITIVITY_HIGH",
                            "prefix_padding_ms": 20,
                            "silence_duration_ms": 100,
                        },
                    },
                }
            }
        )
    )

    # Receive setup response
    raw_response = await ws.recv(decode=False)
    setup_response = json.loads(raw_response.decode("ascii"))

    # Send audio message
    audio_bytes = Path("sample.pcm").read_bytes()
    msg = {
        "realtime_input": {
            "media_chunks": [
                {
                    "data": base64.b64encode(audio_bytes).decode(),
                    "mime_type": "audio/pcm",
                }
            ]
        }
    }

    await ws.send(json.dumps(msg))

    responses = []

    # Receive chucks of server response
    async for raw_response in ws:
        response = json.loads(raw_response.decode())
        server_content = response.pop("serverContent", None)
        if server_content is None:
            break

        model_turn = server_content.pop("modelTurn", None)
        if model_turn is not None:
            parts = model_turn.pop("parts", None)
            if parts is not None:
                responses.append(parts[0]["text"])

        # Interruption
        interrupted = server_content.pop("interrupted", None)
        if interrupted:
            # The model generation was interrupted
            response.append("The session was interrupted")

        # End of turn
        turn_complete = server_content.pop("turnComplete", None)
        if turn_complete:
            break

    # Print the server response
    display(Markdown(f"**Response >** {''.join(responses)}"))

## Using the Gemini 2.5 Flash native audio dialog


Gemini 2.5 Flash with Live API features native audio dialog capabilities. In addition to the standard Live API features, this model includes:

- Enhanced voice quality and adaptability
- Introducing proactive audio
- Introducing affective dialog

In [ ]:
# fmt: off
MODEL_ID = "gemini-live-2.5-flash-preview-native-audio-09-2025"  # @param {type: "string"}
# fmt: on

MODEL = (
    f"projects/{PROJECT_ID}/locations/{LOCATION}/publishers/google/models/{MODEL_ID}"
)

### **Example 1**: Proactive audio


When proactive audio is enabled, the model only responds when it's relevant. The model generates text transcripts and audio responses proactively only for queries directed to the device, and does not respond to non-device directed queries.

In [ ]:
# Set model generation_config
GENERATION_CONFIG = {
    "response_modalities": ["AUDIO"],
}


headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {bearer_token[0]}",
}

# Connect to the server
async with connect(SERVICE_URL, additional_headers=headers) as ws:
    # Set up the session
    await ws.send(
        json.dumps(
            {
                "setup": {
                    "model": MODEL,
                    "generation_config": GENERATION_CONFIG,
                    "input_audio_transcription": {},
                    "output_audio_transcription": {},
                    "proactivity": {"proactive_audio": True},
                }
            }
        )
    )

    # Receive setup response
    raw_response = await ws.recv(decode=False)
    setup_response = json.loads(raw_response.decode("ascii"))

    # Send text message
    text_input = "Hello? Gemini are you there?"
    display(Markdown(f"**Input:** {text_input}"))

    msg = {
        "client_content": {
            "turns": [{"role": "user", "parts": [{"text": text_input}]}],
            "turn_complete": True,
        }
    }

    await ws.send(json.dumps(msg))

    responses = []
    input_transcriptions = []
    output_transcriptions = []

    # Receive chucks of server response
    async for raw_response in ws:
        response = json.loads(raw_response.decode())
        server_content = response.pop("serverContent", None)
        if server_content is None:
            break

        if (
            input_transcription := server_content.get("inputTranscription")
        ) is not None:
            if (text := input_transcription.get("text")) is not None:
                input_transcriptions.append(text)
        if (
            output_transcription := server_content.get("outputTranscription")
        ) is not None:
            if (text := output_transcription.get("text")) is not None:
                output_transcriptions.append(text)

        model_turn = server_content.pop("modelTurn", None)
        if model_turn is not None:
            parts = model_turn.pop("parts", None)
            if parts is not None:
                for part in parts:
                    pcm_data = base64.b64decode(part["inlineData"]["data"])
                    responses.append(np.frombuffer(pcm_data, dtype=np.int16))

        # End of turn
        turn_complete = server_content.pop("turnComplete", None)
        if turn_complete:
            break

    if input_transcriptions:
        display(Markdown(f"**Input transcription >** {''.join(input_transcriptions)}"))

    if responses:
        # Play the returned audio message
        display(Audio(np.concatenate(responses), rate=24000, autoplay=True))

    if output_transcriptions:
        display(
            Markdown(f"**Output transcription >** {''.join(output_transcriptions)}")
        )

### **Example 2**: Affective Dialog

When affective dialog is enabled, the model can understand and respond appropriately to users' emotional expressions for more nuanced conversations.

In [ ]:
# Set model generation_config
GENERATION_CONFIG = {
    "response_modalities": ["AUDIO"],
    "enable_affective_dialog": True,
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {bearer_token[0]}",
}

# Connect to the server
async with connect(SERVICE_URL, additional_headers=headers) as ws:
    # Setup the session
    await ws.send(
        json.dumps(
            {
                "setup": {
                    "model": MODEL,
                    "generation_config": GENERATION_CONFIG,
                }
            }
        )
    )

    # Receive setup response
    raw_response = await ws.recv(decode=False)
    setup_response = json.loads(raw_response.decode("ascii"))

    # Send text message
    text_input = "Hello? Gemini are you there? It's really a good day!"
    display(Markdown(f"**Input:** {text_input}"))

    msg = {
        "client_content": {
            "turns": [{"role": "user", "parts": [{"text": text_input}]}],
            "turn_complete": True,
        }
    }

    await ws.send(json.dumps(msg))

    responses = []

    # Receive chucks of server response
    async for raw_response in ws:
        response = json.loads(raw_response.decode())
        server_content = response.pop("serverContent", None)
        if server_content is None:
            break

        model_turn = server_content.pop("modelTurn", None)
        if model_turn is not None:
            parts = model_turn.pop("parts", None)
            if parts is not None:
                for part in parts:
                    pcm_data = base64.b64decode(part["inlineData"]["data"])
                    responses.append(np.frombuffer(pcm_data, dtype=np.int16))

        # End of turn
        turn_complete = server_content.pop("turnComplete", None)
        if turn_complete:
            break

    # Play the returned audio message
    display(Audio(np.concatenate(responses), rate=24000, autoplay=True))

## What's next


- Try [getting started with the Live API with the Gen AI SDK](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_genai_sdk.ipynb)
- Learn how to [build a web application that enables you to use your voice and camera to talk to Gemini 2.0 through the Live API.](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/gemini/multimodal-live-api/websocket-demo-app)
- See the [Live API reference docs](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-live).
- Explore other notebooks in the [Google Cloud Generative AI GitHub repository](https://github.com/GoogleCloudPlatform/generative-ai).